In [1]:
import requests
import folium
import numpy as np
import pandas as pd

In [2]:
class TrafficFlowFetcher:
    def __init__(self, api_key, 
                 base_url="api.tomtom.com", 
                 version_number=4, 
                 style="absolute", 
                 zoom=22, 
                 form="json", 
                 thickness=1):
        """
        Initializes the TrafficFlowFetcher class with parameters for the TomTom Traffic API.
        """
        self.api_key = api_key
        self.base_url = base_url
        self.version_number = version_number
        self.style = style
        self.zoom = zoom
        self.form = form
        self.thickness = thickness

    def get_traffic_flow_per_cord(self, location):
        """
        Fetches real-time data on road speeds and travel times from the TomTom Traffic API.
        """
        url = f"https://{self.base_url}/traffic/services/{self.version_number}/flowSegmentData/{self.style}/{self.zoom}/{self.form}?key={self.api_key}&point={location}&thickness={self.thickness}"
        response = requests.get(url)
        
        if response.status_code == 200:
            return response.json()
        else:
            return {"error": f"Failed to fetch data. Status code: {response.status_code}"}
    
    def json_to_dataframe(self, json_data):
        """
        Converts JSON data from the TomTom Traffic API into a structured DataFrame.
        """
        if 'flowSegmentData' in json_data:
            data = json_data['flowSegmentData']
            df = pd.DataFrame({
                "currentSpeed": [data['currentSpeed']],
                "freeFlowSpeed": [data['freeFlowSpeed']],
                "currentTravelTime": [data['currentTravelTime']],
                "freeFlowTravelTime": [data['freeFlowTravelTime']],
                "coordinates": [[(coord['latitude'], coord['longitude']) for coord in data['coordinates']['coordinate']]]
            })
            return df
        else:
            return pd.DataFrame({"error": [json_data.get("error", "Unknown error")]})

    def save_df_to_csv(self, df, filename):
        """
        Save the DataFrame to a CSV file.
        """
        df.to_csv(filename, index=False)

    def get_traffic_flow(self, nyc_lat_min, nyc_lat_max, nyc_lon_min, nyc_lon_max, lat_step=0.0005, lon_step=0.0015):
        """
        Generates a grid of latitude and longitude points and fetches traffic data for each.
        Concatenates all data into a single DataFrame.
        """
        lat_intervals = np.arange(nyc_lat_min, nyc_lat_max, lat_step)
        lon_intervals = np.arange(nyc_lon_min, nyc_lon_max, lon_step)
        
        combined_df = pd.DataFrame()  # Initialize an empty DataFrame to store results
        
        for lat in lat_intervals:
            for lon in lon_intervals:
                location = f"{lat},{lon}"
                json_data = self.get_traffic_flow_per_cord(location)
                df = self.json_to_dataframe(json_data)
                combined_df = pd.concat([combined_df, df], ignore_index=True)
        
        return combined_df


In [3]:
def get_color_by_speed(speed):
    """
    Returns color based on speed:
    - Gray: speed <= 1 km/h
    - Red: 1 < speed <= 60 km/h
    - Yellow: 60 < speed <= 120 km/h
    - Green: speed > 120 km/h
    """
    if speed <= 1:
        return "gray"
    elif 1 < speed <= 60:
        return "red"
    elif 60 < speed <= 120:
        return "yellow"
    else:
        return "green"

def generate_traffic_map(flows_df):
    """
    Generates a traffic flow map based on pre-fetched data in flows_df.
    Each row in flows_df is used to add a colored polyline to the map.
    """
    # Initialize map for visualization
    nyc_map = folium.Map(location=[40.8075, -73.9626], zoom_start=14)  # centered near Columbia University

    # Loop through each entry in flows_df to create polylines on the map
    for _, row in flows_df.iterrows():
        speed = row['currentSpeed']
        color = get_color_by_speed(speed)

        # Extract coordinates for the road segment from the DataFrame
        coords = row['coordinates']
        
        # Add colored polyline (representing the road segment) to the map
        folium.PolyLine(
            locations=coords,
            color=color,
            weight=5,
            opacity=0.7
        ).add_to(nyc_map)

    # Save map as an HTML file
    nyc_map.save("columbia_flow_test.html")


In [5]:
api_key = "N8xhZrbfY7NZEqbqIeEedz4xUBmXARBG"
nyc_lat_min, nyc_lat_max = 40.79830, 40.81826    # Latitude from 103rd to 130th Street
nyc_lon_min, nyc_lon_max = -73.97502, -73.94881  # Longitude from the west edge of Manhattan to Central Park West


flow = TrafficFlowFetcher(api_key)
flows_df = flow.get_traffic_flow(nyc_lat_min, nyc_lat_max, nyc_lon_min, nyc_lon_max)
generate_traffic_map(flows_df)